# manueverMotifs: results analysis
## Driver D6 - Lateral acceleration

### 29/09/2019

***

**Contents**

1. [Setup](#section1)
2. [Load motifs](#section2)
3. [Motif results table](#section3)
4. [DBSCAN clustering](#section4)

***

<a id='section1'></a>
## 1. Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import pickle
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from extendedMD.emd import find_motifs_from_emd
from extendedMD.pruning import prune_motifs_with_mdl

# local code
import sys
import os
sys.path.insert(0, os.path.abspath('../scripts'))
import uah_utils
import motif_utils

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
cwd = os.getcwd()
data_folder = os.path.abspath(os.path.join(cwd, os.pardir, os.pardir, 'data-uah/D6'))
output_folder = os.path.abspath(os.path.join(cwd, os.pardir, 'data-motifs'))
trips = ['20151221120051', '20151217164730', '20151217162714', '20151221112434']
R = 0.1
freq_per_second=10

driver_df = uah_utils.get_full_point_uah_data(data_folder, freq_per_second)

<a id='section2'></a>
## 2. Load motifs

In [5]:
lat_motifs = {}
for trip_id in trips:
    trip_df = driver_df[driver_df['trip_id'] == trip_id].reset_index(drop=True)
    ts = trip_df['ay'].values
    file_name = "motif_lat_" + trip_df["trip_label"][0] + "_" + trip_df["road"][0] + ".p"
    motif_dic_list = pickle.load(open(os.path.join(output_folder, file_name), 'rb'))
    lat_motifs[trip_id] = motif_dic_list

In [6]:
lon_motifs = {}
for trip_id in trips:
    trip_df = driver_df[driver_df['trip_id'] == trip_id].reset_index(drop=True)
    ts = trip_df['az'].values
    file_name = "motif_lon_" + trip_df["trip_label"][0] + "_" + trip_df["road"][0] + ".p"
    motif_dic_list = pickle.load(open(os.path.join(output_folder, file_name), 'rb'))
    lon_motifs[trip_id] = motif_dic_list

<a id='section3'></a>
## 3. Motif results table

### Lateral experiment

In [7]:
lat_results_list = []
for trip_id in trips:
    trip_df = driver_df[driver_df['trip_id'] == trip_id].reset_index(drop=True)
    ts = trip_df['ay'].values
    motif_dic_list = lat_motifs[trip_id]
    pruned_motif_dic_list = prune_motifs_with_mdl(ts, motif_dic_list, R)
    results_dic = {
        "motif": trip_df["trip_label"][0] + "-" + trip_df["road"][0],
        "n_motifs": len(motif_dic_list),
        "n_prunned_motifs": len(pruned_motif_dic_list),
        "lc_detection_rate": motif_utils.compute_lc_detection_rate(pruned_motif_dic_list, trip_df),
        "turn_detection_rate": motif_utils.compute_turn_detection_rate(pruned_motif_dic_list, trip_df),
    }
    lat_results_list.append(results_dic)
pd.DataFrame(lat_results_list, columns=["motif", "n_motifs", "n_prunned_motifs", "lc_detection_rate", "turn_detection_rate"])

,motif,n_motifs,n_prunned_motifs,lc_detection_rate,turn_detection_rate
0,aggressive-motorway,634,7,0.576923,0.290323
1,drowsy-motorway,1464,8,0.285714,0.360000
2,normal-motorway,1532,3,0.437500,0.181818
3,normal-secondary,1113,1,NaN,0.250000


### Longitudinal experiment

In [8]:
lon_results_list = []
for trip_id in trips:
    trip_df = driver_df[driver_df['trip_id'] == trip_id].reset_index(drop=True)
    ts = trip_df['az'].values
    motif_dic_list = lon_motifs[trip_id]
    pruned_motif_dic_list = prune_motifs_with_mdl(ts, motif_dic_list, R)
    results_dic = {
        "motif": trip_df["trip_label"][0] + "-" + trip_df["road"][0],
        "n_motifs": len(motif_dic_list),
        "n_prunned_motifs": len(pruned_motif_dic_list),
        "acc_detection_rate": motif_utils.compute_acc_detection_rate(pruned_motif_dic_list, trip_df),
        "brake_detection_rate": motif_utils.compute_brake_detection_rate(pruned_motif_dic_list, trip_df),   
    }
    lon_results_list.append(results_dic)
pd.DataFrame(lon_results_list, columns=["motif", "n_motifs", "n_prunned_motifs", "acc_detection_rate", "brake_detection_rate"])

,motif,n_motifs,n_prunned_motifs,acc_detection_rate,brake_detection_rate
0,aggressive-motorway,1272,8,0.285714,0.608696
1,drowsy-motorway,2039,5,0.200000,0.250000
2,normal-motorway,1849,3,0.400000,0.400000
3,normal-secondary,1591,3,0.000000,0.000000


<a id='section4'></a>
## 4. DBSCAN clustering

In [15]:
min_samples = 15
eps = R/2

### Lateral experiment

In [16]:
lat_dbscan_list = []
for trip_id in trips:
    trip_df = driver_df[driver_df['trip_id'] == trip_id].reset_index(drop=True)
    file_name = "dist_lat_" + trip_df["trip_label"][0] + "_" + trip_df["road"][0] + ".npy"
    center_dist_mat = np.load(os.path.join(output_folder, file_name))
    dbscan_model = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed').fit(center_dist_mat)
    cluster_labels = dbscan_model.labels_
    results_dic = {
        "motif": trip_df["trip_label"][0] + "-" + trip_df["road"][0],
        "n_clusters": len(set(cluster_labels) - {-1}),
        "n_outliers": sum(cluster_labels == -1)
    }
    lat_dbscan_list.append(results_dic)
pd.DataFrame(lat_dbscan_list, columns=["motif", "n_clusters", "n_outliers"])

,motif,n_clusters,n_outliers
0,aggressive-motorway,1,614
1,drowsy-motorway,5,1292
2,normal-motorway,6,1328
3,normal-secondary,2,929


### Longitudinal experiment

In [17]:
lon_dbscan_list = []
for trip_id in trips:
    trip_df = driver_df[driver_df['trip_id'] == trip_id].reset_index(drop=True)
    file_name = "dist_lon_" + trip_df["trip_label"][0] + "_" + trip_df["road"][0] + ".npy"
    center_dist_mat = np.load(os.path.join(output_folder, file_name))
    dbscan_model = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed').fit(center_dist_mat)
    cluster_labels = dbscan_model.labels_
    results_dic = {
        "motif": trip_df["trip_label"][0] + "-" + trip_df["road"][0],
        "n_clusters": len(set(cluster_labels) - {-1}),
        "n_outliers": sum(cluster_labels == -1)
    }
    lon_dbscan_list.append(results_dic)
pd.DataFrame(lon_dbscan_list, columns=["motif", "n_clusters", "n_outliers"])

,motif,n_clusters,n_outliers
0,aggressive-motorway,8,771
1,drowsy-motorway,5,477
2,normal-motorway,5,383
3,normal-secondary,2,162
